In [38]:
#from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import numpy as np
#import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import logging
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk
import json

from gensim.models import word2vec as w2v
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /Users/bray/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
#with open('/Users/bray/Desktop/FOMC/FOMC.json', 'rb') as f:
    #data = f.readlines()
    
#data2 = json.dumps(data)
#print(data)
# remove the trailing "\n" from each line
#data = '\n'.join(map(lambda x: x.decode('utf8', 'ignore')changed_project_data['rdars']))

#data = data.decode("utf-8") 

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
#data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
#data_df = pd.read_json(data)
#print(data)

In [40]:
def tokenize_sentences(sentences):
    ret = []
    max_s = len(sentences)
    print("Got " + str(max_s) + " sentences.")
    for count, s in enumerate(sentences):
        tokens = []
        words = re.split(r'(\s+)', s)
        if len(words) > 0:
            for w in words:
                if w is not None:
                    w = w.strip()
                    w = w.lower()
                    if w.isspace() or w == "\n" or w == "\r":
                        w = None
                    if len(w) < 1:
                        w = None
                    if w is not None:
                        tokens.append(w)
        if len(tokens) > 0:
            ret.append(tokens)
        if count % 50 == 0:
            print_progress(count, max_s)
    return ret

In [41]:
def clean_sentences(tokens):
    all_stopwords = load_json("stopwords-iso.json")
    extra_stopwords = ["to", "lle", "h.", "oo", "on", "muk", "kov", "km", "ia", "täm", "sy", "but", ":sta", "hi", "py", "xd", "rr", "x:", "smg", "kum", "uut", "kho", "k", "04n", "vtt", "htt", "the", "kin", "#8", "van", "tii", "lt3", "g", "ko", "ett", "mys", "tnn", "hyv", "tm", "mit", "tss", "siit", "pit", "viel", "of", "n", "saa", "tll", "eik", "nin", "nii", "t", "tmn", "lsn", "j", "miss", "pivn", "yhn", "mik", "tn", "tt", "sek", "lis", "mist", "tehd", "sai", "l", "thn", "mm", "k", "ku", "s", "hn", "nit", "s", "no", "m", "ky", "tst", "mut", "nm", "y", "lpi", "siin", "a", "in", "ehk", "h", "e", "piv", "oy", "p", "yh", "sill", "min", "o", "va", "el", "tyn", "na", "the", "tit", "to", "iti", "tehdn", "tlt", "ois", ":", "v", "?", "!", "&","//","href","\\","``"]
    stopwords = None
    if all_stopwords is not None:
        stopwords = all_stopwords["fi"]
        stopwords += extra_stopwords
    ret = []
    max_s = len(tokens)
    for count, sentence in enumerate(tokens):
        if count % 50 == 0:
            print_progress(count, max_s)
        cleaned = []
        for token in sentence:
            if len(token) > 0:
                if stopwords is not None:
                    for s in stopwords:
                        if token == s:
                            token = None
                if token is not None:
                    if re.search("^[0-9\.\-\s\/]+$", token):
                        token = None
                if token is not None:
                    cleaned.append(token)
            if len(cleaned) > 0:
                ret.append(cleaned)
    return ret

In [42]:
def get_word_frequencies(corpus):
    frequencies = Counter()
    for sentence in corpus:
        for word in sentence:
            frequencies[word] += 1
    freq = frequencies.most_common()
    return freq


In [43]:
def try_load_or_process(filename, processor_fn, function_arg):
    load_fn = None
    save_fn = None
    if filename.endswith("json"):
        load_fn = load_json
        save_fn = save_json
    else:
        load_fn = load_bin
        save_fn = save_bin
        
    if os.path.exists(filename):
        return load_fn(filename)
    else:
        ret = processor_fn(function_arg)
        save_fn(ret, filename)
        return ret

def print_progress(current, maximum):
    sys.stdout.write("\r")
    sys.stdout.flush()
    sys.stdout.write(str(current) + "/" + str(maximum))
    sys.stdout.flush()

def save_bin(item, filename):
    with open(filename, "wb") as f:
        cPickle.dump(item, f)

def load_bin(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return cPickle.load(f)

def save_json(variable, filename):
    with io.open(filename, "w", encoding="utf-8") as f:
        f.write(unicode(json.dumps(variable, indent=4, ensure_ascii=False)))

def load_json(filename):
    ret = None
    if os.path.exists(filename):
        try:
            with io.open(filename, "r", encoding="utf-8") as f:
                ret = json.load(f)
        except:
            pass
    return ret

In [44]:
def process_raw_data(input_file):
    valid = u"0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ#@.:/ äöåÄÖÅ"
    url_match = "(https?:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]+\.?[0-9a-zA-Z\-\_]*\/?.*)"
    name_match = "\@[\_0-9a-zA-Z]+\:?"
    lines = []
    print("Loading raw data from: " + input_file)
    if os.path.exists(input_file):
        with io.open(input_file, 'r', encoding="utf-8") as f:
            lines = f.readlines()
    num_lines = len(lines)
    ret = []
    for count, text in enumerate(lines):
        if count % 50 == 0:
            print_progress(count, num_lines)
        text = re.sub(url_match, u"", text)
        text = re.sub(name_match, u"", text)
        text = re.sub("\&amp\;?", u"", text)
        text = re.sub("[\:\.]{1,}$", u"", text)
        text = re.sub("^RT\:?", u"", text)
        text = u''.join(x for x in text if x in valid)
        text = text.strip()
        if len(text.split()) > 5:
            if text not in ret:
                ret.append(text)
    return ret

In [45]:
def get_word2vec(sentences):
    num_workers = multiprocessing.cpu_count()
    num_features = 200
    epoch_count = 10
    sentence_count = len(sentences)
    w2v_file = os.path.join(save_dir, "word_vectors.w2v")
    word2vec = None
    if os.path.exists(w2v_file):
        print("w2v model loaded from " + w2v_file)
        word2vec = w2v.Word2Vec.load(w2v_file)
    else:
        word2vec = w2v.Word2Vec(sg=1,
                                seed=1,
                                workers=num_workers,
                                size=num_features,
                                min_count=6,
                                window=5,
                                sample=0)
        print("Building vocab...")
        word2vec.build_vocab(sentences)
        print("Word2Vec vocabulary length:", len(word2vec.wv.vocab))
        print("Training...")
        word2vec.train(sentences, total_examples=sentence_count, epochs=epoch_count)
        print("Saving model...")
        word2vec.save(w2v_file)
    return word2vec

In [46]:
if __name__ == '__main__':
    
    save_dir = 'analysis'
    fname = 'FOMC.json'
    with open(fname) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content]
    sentence = []
    
    for x in content:
        nltk_tokens = nltk.word_tokenize(x)
        sentence.append(nltk_tokens)
        
        
    print("Cleaning tokens")    
    cleaned = clean_sentences(sentence)
    
    print("Getting word frequencies")
    frq = get_word_frequencies(cleaned)
    ss = len(frq)
    print("Unique words: " + str(ss))
    
    print("Instantiating word2vec model")
    word2vec = get_word2vec(sentence)
    vocab = list(word2vec.wv.vocab.keys())
    vocab_len = len(vocab)
    print("word2vec vocab contains " + str(vocab_len) + " items.")
    dim01 = word2vec.wv[vocab[0]].shape[0]
    print("word2vec items have " + str(dim01) + " features.")

Cleaning tokens
500/528Getting word frequencies
Unique words: 1063
Instantiating word2vec model
w2v model loaded from analysis/word_vectors.w2v
word2vec vocab contains 555 items.
word2vec items have 200 features.


In [47]:
import pprint as pp

In [48]:
pp.pprint(word2vec.wv.most_similar('rates', topn=10))

[('interest', 0.736855149269104),
 ('paid', 0.7104199528694153),
 ('through', 0.706190824508667),
 ('rate', 0.6989840269088745),
 ('lower', 0.6951702833175659),
 ('establishment', 0.6948161721229553),
 ('primary', 0.6815010905265808),
 ('stayed', 0.6791510581970215),
 ('required', 0.6698943376541138),
 ('credit', 0.6663016080856323)]


In [51]:
from gensim.models.word2vec import Word2Vec   
import pickle
w2v_file = os.path.join(save_dir, "word_vectors.w2v")
model = Word2Vec.load(w2v_file)  #WORD_vector.w2v是我已经生成的模型
 
print(model.wv.index2word)    #獲得所有的詞彙

['the', ',', 'of', 'to', 'and', '.', '``', 'in', 'Committee', "''", 'Federal', 'a', 'rate', ';', 'inflation', 'percent', 'that', 'Reserve', 'will', 'at', 'The', 'for', 'securities', '<', '>', 'its', 'market', 'policy', 'Open', 'Market', 'federal', 'funds', "'s", 'on', 'economic', 'agency', 'conditions', '2', 'range', 'labor', 'has', 'as', 'operations', 'monetary', 'by', ':', 'mortgage-backed', 'target', 'with', 'are', 'Desk', 'from', 'expected', '/a', 'href=\\', 'is', 'In', 'directs', 'System', 'information', 'Treasury', 'measures', 'necessary', 'Board', 'over', 'payments', 'be', '[', ']', 'holdings', '$', 'billion', 'principal', 'maintain', 'action', 'This', 'developments', 'stance', 'continue', 'employment', 'Governors', 'activity', 'have', 'been', 'remain', 'expectations', 'maximum', 'objective', 'including', 'financial', 'further', 'implement', 'outlook', 'repurchase', 'regarding', 'received', 'recent', 'or', 'New', 'York', 'voted', 'reverse', 'adjustments', 'open', 'day', 'indicat

In [52]:
#下方是所有詞彙之個別詞向量
for word in model.wv.index2word:
    print(word,model[word]) 

/Users/bray/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


the [-2.22173840e-01  2.31747463e-01 -1.56110853e-01 -1.77926451e-01
  4.92997281e-02 -1.31311938e-01 -1.31421596e-01 -4.39727604e-02
  8.95155966e-02  1.02593796e-02 -1.71075854e-02  2.30789706e-02
 -3.36793363e-01  6.76116124e-02 -4.26895509e-04  1.44593135e-01
 -1.42649924e-02  1.02386646e-01 -5.67417324e-01  3.65242988e-01
  2.37266928e-01 -9.30521637e-02  2.27640510e-01 -2.27176957e-02
  1.15482837e-01 -2.41060525e-01 -1.26288921e-01  8.39846507e-02
  2.37023388e-03  2.81509131e-01 -1.57403976e-01  1.39485449e-01
  1.73521519e-01 -1.28228158e-01  1.95480347e-01  2.63948321e-01
  7.99588785e-02 -1.86116174e-01  1.64977372e-01 -2.96295941e-01
  4.02374864e-02  1.52663970e-02  6.80535436e-02 -1.88346177e-01
  3.16231161e-01  7.91803896e-02  1.70322303e-02  5.64899258e-02
 -3.47096384e-01  5.07565625e-02  4.19211894e-01 -4.86538261e-02
 -1.01857573e-01 -1.73687562e-01  1.07500635e-01  8.19827765e-02
 -2.96527416e-01 -2.31682941e-01  2.78473645e-01 -7.56802410e-02
  9.14212465e-02  2.3

 -3.22372526e-01  3.21204096e-01 -3.28019410e-01 -2.20192820e-01]
this [-0.26689336  0.09722053 -0.15812543  0.15895234 -0.03091375 -0.29539928
  0.23169596 -0.38185456  0.09155413  0.20708884 -0.08268796  0.01141553
 -0.40570146 -0.07143003  0.06352576  0.07854741 -0.17260514 -0.07478788
 -0.11254282  0.15718366  0.15946862 -0.13801576  0.2081909   0.02590053
 -0.19569738 -0.21477613 -0.14065386  0.02035036 -0.11937907  0.15526503
  0.03620948  0.21244086  0.16531396 -0.24461088 -0.03499795 -0.11509103
  0.18365477 -0.2683898  -0.3403412   0.08357833 -0.08817452  0.08938596
  0.09704114  0.12397213  0.17651954  0.05321801  0.5699288   0.04901408
 -0.11184809 -0.14567006  0.2588823   0.13638403 -0.20229615 -0.09063777
  0.01142716 -0.00778686 -0.3051256   0.23039933  0.13124453  0.020504
  0.05238012  0.00062701  0.08162604  0.3706277   0.25284958  0.20987576
 -0.29187402 -0.01030985  0.23243412 -0.19965076  0.09945087 -0.08503554
  0.14802454 -0.0925035  -0.15968786 -0.06929681 -0.286

wide [-0.4539361   0.10619885 -0.12436655  0.48224983 -0.7553808  -0.46330422
  0.2366497  -0.2321416   0.06881933  0.512394    0.06538346  0.42990842
 -0.2394893   0.00287811  0.04834183  0.4509259   0.51136833  0.06556953
 -0.22745784  0.32380065 -0.07485995  0.11701866 -0.4975157   0.2606984
  0.33143485 -0.6705453   0.38071638  0.00268025  0.08308166  0.45026776
  0.35852006 -0.5495715   0.3122142  -0.08216062 -0.4356653  -0.10270394
  0.00495493 -0.27218923  0.31180295 -0.4077973  -0.09747712 -0.0909053
 -0.17050372 -0.46633378  0.46041387 -0.03705329  0.00266782 -0.3175576
 -0.41094986  0.4941135   0.2935507  -0.49345592  0.12039788 -0.552797
  0.06739893 -0.19543807 -0.06950721 -0.57019186 -0.24152738 -0.20414129
  0.647339    0.01460864  0.8459038   0.26425982  0.08170069  0.4191511
 -0.42462507  0.21992736  0.45583725  0.0349316  -0.5508576  -0.14401892
  0.1423518  -0.14296539 -0.15668951  0.29927868  0.12797181 -0.3998756
 -0.22571225 -0.42546523  0.4762286   0.2588589  -0.1

However [-1.42091990e-01  5.47742136e-02  8.69282633e-02  2.38352731e-01
  1.15571834e-01 -2.36445218e-02 -6.46375641e-02 -4.73223627e-01
  8.31320509e-02 -1.21243715e-01  2.01648340e-01 -5.45736104e-02
 -4.42061484e-01 -2.41529107e-01 -4.98137549e-02  2.76773095e-01
 -2.80471649e-02  3.27927712e-03 -6.75961450e-02  3.39928895e-01
  2.06949651e-01  1.06638387e-01  2.18581378e-01  4.10168231e-01
  1.32826433e-01 -2.61544764e-01 -1.16905324e-01 -4.42893058e-02
  1.61194578e-02  1.23195067e-01 -1.43843563e-02  1.64609565e-03
  1.44986704e-01 -1.65623307e-01  1.69226438e-01  4.92391177e-02
  1.40627652e-01 -6.27762079e-01 -2.68996924e-01 -8.07214454e-02
  8.30644891e-02 -2.02429622e-01 -3.90414074e-02  2.00923890e-01
  2.12069333e-01  8.20232853e-02  8.89084414e-02 -2.67853737e-01
 -3.04536551e-01 -9.91980657e-02  1.56594872e-01  5.70532419e-02
 -9.32409018e-02 -1.17974401e-01 -1.29451856e-01  4.34221141e-02
 -2.55930126e-01  1.38558164e-01  2.85830736e-01 -1.79431945e-01
  2.13778153e-01 

Policy [-0.17531206  0.2646728  -0.1583585  -0.02533563 -0.03997767 -0.04914071
  0.16838205 -0.11704971 -0.23416063  0.13324325 -0.08644701  0.33287674
 -0.15304907  0.36406052 -0.01455068  0.10072669  0.01394075  0.01272994
 -0.05393723  0.1524487   0.42266864 -0.25039247  0.20543033  0.03485804
  0.00616591 -0.3278527  -0.29139832 -0.14968431 -0.05678559  0.20364757
  0.14385825 -0.14688739  0.17170882 -0.1985634  -0.26742658  0.12143127
  0.02391265  0.20619589 -0.20373692 -0.16085139  0.25933304  0.04688463
 -0.24280243 -0.2743784   0.12858948 -0.02580754 -0.02355322 -0.05831465
 -0.14840956  0.15470886  0.13745455 -0.13065775  0.09472064 -0.2025189
  0.16126046  0.11942315  0.03504073  0.041346   -0.11174548  0.15026426
 -0.1716095  -0.01026924  0.04224223 -0.03296718  0.09333543 -0.04718161
  0.13477837 -0.06401309  0.01158303  0.10288648 -0.13878162 -0.03253542
  0.02519573 -0.28313652  0.06125359 -0.1861526  -0.07672254 -0.20818125
 -0.02402476 -0.2801574  -0.06718126  0.11035

I. [-0.04643878 -0.07865011 -0.22255397  0.08290317 -0.19941199 -0.07139769
  0.70755106 -0.19996402 -0.03601494  0.5914251  -0.31614715  0.27326363
 -0.23479646 -0.0697908  -0.2078768   0.4242866  -0.21044345  0.01737596
 -0.29060188 -0.06766983 -0.07127514 -0.07532793  0.17908852  0.2347751
  0.10565183 -0.18944941 -0.12455236  0.0086331  -0.08160602  0.11148079
  0.20243981  0.00535097  0.47538882 -0.36313    -0.4099117  -0.12496287
  0.08527088 -0.42921823 -0.22850744 -0.49798006  0.17125943  0.15283695
 -0.01993559 -0.08575583  0.23021968 -0.09465346  0.44806355  0.12466516
 -0.1867477   0.20374699  0.25276068 -0.07998581  0.03508554 -0.25347194
 -0.0477016   0.01095158 -0.09527738 -0.08176454 -0.2305047  -0.06919315
 -0.01689028  0.17963298  0.15512265 -0.34001213  0.09710293  0.0898681
  0.08777653 -0.07841659 -0.0578726   0.16489047 -0.15420856  0.03401532
  0.32872957 -0.04774358 -0.21595983  0.24030243 -0.03298961 -0.0388772
 -0.03708729  0.02024218 -0.05756275  0.13543428  0